# Script em python para ordenar os melhores resultados das tabelas xlsx, para verificar quais foram as melhores abordagens

In [12]:
import pandas as pd
import os
from glob import glob

# Diretório base (modifique conforme necessário)
base_directory = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\abordagem2\saida\4-escolha\selectKbest\75'
output_txt_file = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\processamento\ordena-resultados\8-ordenacao_modelos_selectkbest-75.txt'

# Lista para armazenar os dados combinados
all_data = []

# Busca recursiva por TODOS os arquivos .xlsx nas subpastas
for excel_file in glob(os.path.join(base_directory, '**/*.xlsx'), recursive=True):
    try:
        # Lê o arquivo Excel
        df = pd.read_excel(excel_file)
        
        # Adiciona metadados do arquivo
        df['file_path'] = excel_file  # Caminho completo
        
        # Extrai nome do dataset do caminho (exemplo: "maxwell", "cocomo81")
        dataset = next((d for d in ['maxwell', 'cocomo81', 'desharnais', 'china'] 
                        if d in excel_file.lower()), 'outro')
        df['dataset'] = dataset
        
        # Adiciona à lista principal
        all_data.append(df)
        
    except Exception as e:
        print(f"Erro ao processar {excel_file}: {str(e)}")

# Combina todos os dados em um único DataFrame
combined_df = pd.concat(all_data, ignore_index=True)

# Função para gerar relatórios de ordenação
def generate_ranking_report(df, metric, ascending=True):
    return df.sort_values(by=metric, ascending=ascending)['file_path'].unique()

# Gera relatório completo
with open(output_txt_file, 'w') as report:
    for dataset in combined_df['dataset'].unique():
        dataset_df = combined_df[combined_df['dataset'] == dataset]
        
        report.write(f"\n{'='*50}\nDataset: {dataset.upper()}\n{'='*50}\n")
        
        # Metricas para ordenação
        metrics_config = {
            'MSE (Ascendente)': ('mse', True),
            'RMSE (Ascendente)': ('rmse', True),
            'MAE (Ascendente)': ('mae', True),
            'R² (Descendente)': ('r2', False)
        }
        
        for title, (metric, order) in metrics_config.items():
            ranked_files = generate_ranking_report(dataset_df, metric, order)
            report.write(f"\n{title}:\n")
            for rank, path in enumerate(ranked_files[:10], 1):  # Top 10
                report.write(f"{rank}. {os.path.relpath(path, base_directory)}\n")

print("Processamento concluído! Relatório salvo em:", output_txt_file)

Processamento concluído! Relatório salvo em: C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\processamento\ordena-resultados\8-ordenacao_modelos_selectkbest-75.txt


# Pega as melhores abordagens descritas no processo anterior e encontra a melhor, reuni todas em uma única tabela

In [14]:
import pandas as pd
import os
from glob import glob

# Diretório base (modifique conforme necessário)
base_directory = r'C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal'
output_excel_file = os.path.join(base_directory, 'combined_results_analysis.xlsx')

# Subdiretórios de interesse
subdirs = [
    '1-tratamento',
    '2-MIMIC-25', '2-MIMIC-75', '2-GAN-75', '2-GAN-50',
    '3-simulacao\GAN-75', '3-simulacao\MIMIC-25', '3-simulacao\MIMIC-50', '3-simulacao\MIMIC-75'
]

# Lista para armazenar os dados combinados
all_data = []

# Busca recursiva por arquivos relevantes em cada subdiretório
for subdir in subdirs:
    full_path = os.path.join(base_directory, subdir)
    for excel_file in glob(os.path.join(full_path, '**/*.xlsx'), recursive=True):
        try:
            # Lê o arquivo Excel
            df = pd.read_excel(excel_file)
            
            # Adiciona metadados do arquivo
            df['file_path'] = excel_file  # Caminho completo
            df['subdir'] = subdir  # Subdiretório relativo
            
            # Extrai nome do dataset do caminho (exemplo: "maxwell", "cocomo81")
            dataset = next((d for d in ['maxwell', 'cocomo81', 'desharnais', 'china'] 
                            if d in excel_file.lower()), 'outro')
            df['dataset'] = dataset
            
            # Adiciona à lista principal
            all_data.append(df)
            
        except Exception as e:
            print(f"Erro ao processar {excel_file}: {str(e)}")

# Salva a tabela combinada em um único arquivo Excel
combined_df.to_excel(output_excel_file, index=False)

# Análise de melhores abordagens
metrics = ['mse', 'rmse', 'mae', 'r2']
results_summary = {}

for metric in metrics:
    try:
        # Para cada métrica, encontra a abordagem com o melhor valor
        if metric in combined_df.columns:
            best_row = combined_df.loc[combined_df[metric].idxmin()] if metric != 'r2' else combined_df.loc[combined_df[metric].idxmax()]
            results_summary[metric] = best_row[['dataset', 'subdir', 'file_path', metric]]
    except Exception as e:
        print(f"Erro ao analisar a métrica {metric}: {str(e)}")

# Salva os resultados resumidos também no Excel
summary_df = pd.DataFrame.from_dict(results_summary, orient='index')
summary_file = os.path.join(base_directory, 'metrics_summary_analysis.xlsx')
summary_df.to_excel(summary_file)

print("Processamento concluído! Dados combinados e análise de melhores abordagens salvos em:")
print(f"- Dados combinados: {output_excel_file}")
print(f"- Resumo de métricas: {summary_file}")


Erro ao analisar a métrica mse: "['subdir'] not in index"
Erro ao analisar a métrica rmse: "['subdir'] not in index"
Erro ao analisar a métrica mae: "['subdir'] not in index"
Erro ao analisar a métrica r2: "['subdir'] not in index"
Processamento concluído! Dados combinados e análise de melhores abordagens salvos em:
- Dados combinados: C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\combined_results_analysis.xlsx
- Resumo de métricas: C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\metrics_summary_analysis.xlsx
